<a href="https://colab.research.google.com/github/https://github.com/Hackathorn/WILS-NEURIPS22/blob/master/notebooks/WILS-NEURIPS22-DataPrep-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detection of Localized Manifords

...Using NEURIPS22 Paper Abstracts. Derived from the Cohere Dashboard at https://dashboard.cohere.ai/playground/embed. A sample of 33 NEURIPS22 accepted papers (title + abstract) were extracted.

## Configure

In [1]:
%pip install -Uqqq cohere

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cohere

## Load and embed data

In [6]:
# upload repo locally
!rm -rf /content/WILS-NEURIPS22
!git clone https://github.com/Hackathorn/WILS-NEURIPS22.git

Cloning into 'WILS-NEURIPS22'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 40 (delta 12), reused 38 (delta 10), pack-reused 0
Unpacking objects: 100% (40/40), 462.63 KiB | 8.73 MiB/s, done.


In [10]:
# create list of abstracts to embed
with open("/content/WILS-NEURIPS22/doc/NEURIPS22-abstract-sample-UTF8.txt",
          encoding='utf-8', errors='ignore') as f:
    abstracts = f.readlines()
# ...plus their paper titles
titles = [a.split('.')[0].strip() for a in abstracts]

In [14]:
titles[:10]


['"On Kernel-Target Alignment',
 '"On the Convergence of Prior-Guided Zeroth-Order Optimization Algorithms',
 '"Communication Complexity of Distributed Convex Learning and Optimization',
 'Straggler Mitigation in Distributed Optimization Through Data Encoding',
 'Finite Sample Convergence Rates of Zero-Order Stochastic Optimization Methods',
 'Efficient Pure Exploration in Adaptive Round model',
 'On Top-k Selection in Multi-Armed Bandits and Hidden Bipartite Graphs',
 'MaxGap Bandit: Adaptive Algorithms for Approximate Ranking',
 'A Smoothed Analysis of the Greedy Algorithm for the Linear Contextual Bandit Problem',
 'Identifying Outlier Arms in Multi-Armed Bandit']

In [ ]:
# generate embeddings for abstracts
co = cohere.Client('9UQ2FJ61TaGMhZKBoiY7JeGa23qlEyPqmpNjER97') # This is your trial API key
response = co.embed(
  model='embed-english-v2.0',
  texts=abstracts,
)
# convert to np.array
embeddings = np.array(response.embeddings)
embeddings.shape

In [ ]:
# prompt: show stats on embeddings

print(embeddings.shape)
print(embeddings.min())
print(embeddings.max())
print(embeddings.mean())
print(embeddings.std())


(33, 4096)
-10.0703125
9.1640625
-0.00023897772487625735
1.2917951632081166


In [ ]:
!pip install -qq umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# prompt: Using UMAP, find the clusters in embeddings

import umap

reducer = umap.UMAP()
embedding_umap = reducer.fit_transform(embeddings)


In [ ]:
# prompt: plot the clusters in embedding_map

plt.scatter(*embedding_umap.T, c=response.cluster_ids, cmap='tab10')
plt.title('UMAP visualization of document clusters')
plt.show()

AttributeError: ignored